In [ ]:
!pip install sentence_transformers

In [ ]:
! pip install faiss-cpu

In [ ]:
!unzip /content/final_train_dataset.csv.zip

In [5]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
from tqdm.notebook import tqdm

#### Getting data

In [7]:
df = pd.read_csv('/content/final_train_dataset (1).csv', on_bad_lines = 'warn')
df.head()

,Unnamed: 0,context,question,answer
0,0,Она меня бьёт если не включаю Нет Нет)) Нет- с...,Девушки вопрос немного эротичный... смотрите л...,реала вполне хватает)) смотрят.. но отмазывают...
1,1,"верю.. Верю, но сдается мне живет он, где то н...","Вы хоть не верите в сказку, что каждого ждет г...","Тем более, что личный солидный опыт доказывает..."
2,2,"Куда? главное, шо не меня, так что да. это лес...",Я видел на остановке девушка целует другую дев...,"главное, шо не меня, так что да. это лесби нор..."
3,3,на Прочие взаимоотношения расскажи ) ❤❤❤❤❤❤❤❤❤...,девчат вы хоть понимаете на что подписываетесь?,на Прочие взаимоотношения расскажи ) ❤❤❤❤❤❤❤❤❤...
4,4,"Дружбы не существует, есть лишь взаимовыгодные...",бывает ли дружба между маленькой 15 летней дев...,"Дружбы не существует, есть лишь взаимовыгодные..."


In [ ]:
# def get_documents(data: pd.DataFrame) -> list:
#     result = []
#     answers = data['question'].to_list()
#     for i in range(len(answers)):
#         try:

#             result.extend(eval(answers[i]))
#         except SyntaxError as e:
#             continue
#         except NameError as e:
#             continue

#     return result

In [ ]:
# answers = list(set(get_documents(df)))

In [ ]:
# long_answers = []
# for i in answers:
#     if len(i) > 50:
#         long_answers.append(i)

In [8]:
questions = df['question'].to_list()

In [9]:
document_mapper = dict(zip(list(range(len(questions))), questions))

#### Creating index

In [10]:
class Embedding_model:
    def __init__(self):
        self.transformer = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased', device=f"cuda:0")

    def __call__(self, text_batch):
        embeddings = self.transformer.encode(
            text_batch,
            batch_size=100,
            device=f"cuda:0",
        )

        return embeddings

In [11]:
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):

  def __init__(self, document_mapper: dict):
    self.answers = list(document_mapper.values())
    self.indexes = list(document_mapper.keys())

  def __len__(self):
    return len(self.answers)

  def __getitem__(self, idx):
    return self.answers[idx], self.indexes[idx]

In [12]:
def creating_index(document_mapper):
    dataset = MyDataset(document_mapper)
    data_loader = DataLoader(dataset, batch_size=1000, shuffle=True)

    model = Embedding_model()

    base_index = faiss.IndexFlat(512)

    index = faiss.IndexIDMap2(base_index)

    for question, ids in tqdm(data_loader):
        vectors = model(question)

        index.add_with_ids(vectors, ids)

    faiss.write_index(index, f'faiss_index')

    return index

In [ ]:
index = creating_index(document_mapper)

#### Retrieving

In [51]:
#index = faiss.read_index('faiss_index')

In [52]:
model = Embedding_model()

In [53]:
def search(question, k_index=100): #k_mmr=10, diversity=0.1):
    result_dict = {}

    query_emb = np.array([model(question)])
    D, I = index.search(query_emb, k=k_index)
    D, I = list(D[0]), list(I[0])

    vectors = []
    for i in range(len(I)):
        result_dict[D[i]] = document_mapper[I[i]]

    # vectors = index.reconstruct_batch(I)
    # mmr_dict = mmr(query_emb, vectors, list(result_dict.keys()), diversity, k_mmr)

    return result_dict #, mmr_dict

In [54]:
def retrieve(answers: dict, mapper: dict):
  keys = list(answers.keys())
  if keys[0] <= 0.1:
    threshold = keys[0]
    relevants = [answers[key] for key in keys if key < threshold+0.08]
  else:
    relevents = keys[:4]

  ids = []
  for i in relevants:
    ids.append(list(document_mapper.keys())[list(document_mapper.values()).index(i)])

  return list(set(ids))


In [55]:
def nicely_retrieved(question):
  indexes = retrieve(search('Как справиться с депрессией?'), document_mapper)
  return ';'.join(list(df.iloc[indexes]['context']))

In [56]:
print(nicely_retrieved('Как справиться с депрессией?'))

Бабу найди. почему бы не навязать себе депрессию, а потом ныть в тырнетике посмотри порно, подрочи и ложись спать Начать что-то делать, а не искать отговорки чтобы поныть в тырнете Лучший антидепрессант "ДАИХУЙСНИМ" "ДАИХУЙСНИМ" сходи к психотерапевту Все начинается с нашего шоу бизнеса, с их быдлячей музыкой, песен, причем ворованной ))) меняй вкусы, стимулы, увлечения, друзей если тупые!;найди другую девушку, забудешь про неё моментально 100% 2 месяца мало. Время лечит. Со временем боль поугаснет. Не ходи той дорогой, болван. Это же логично Переключись на парней. Может новую завести??;менять обстановку движением Менять обстановку . Много работать . Противположным полом Я салом лечусь. ДЕФИЦИТ МАГНИЯ В ОРГАНИЗМЕ ВЫЗЫВАЕТ ДЕПРЕССИЮ.... ПРОСТО ПРОПЕЙ ПРЕПАРАТ МАГНЕЛИС В6 ИЛИ БОЛЕЕ ДОРОГОЙ АНАЛОГ ИЗ ФРАНЦИИ МАГНЕ В6 Принять радикальные меры. Делать то, что раньше не делала. Чтобы почувствовать риск. Вы увидите, что повысились до нового уровня, и будете гордиться собой! Антидепрессантами 

In [ ]:
# from sentence_transformers.util import dot_score

# def mmr(query_embedding: np.ndarray,
#         reviews_embeddings: np.ndarray,
#         reviews,
#         diversity: float = 0.1,
#         top_n: int = 10):
#     """ Maximal Marginal Relevance
#     Arguments:
#         query_embedding: The document embeddings
#         reviews_embeddings: The embeddings of the selected candidate keywords/phrases
#         reviews: The selected candidate keywords/keyphrases
#         diversity: The diversity of the selected embeddings.
#                    Values between 0 and 1.
#         top_n: The top n items to return
#     Returns:
#             List[str]: The selected keywords/keyphrases
#     """


#     reviews_query_similarity = dot_score(reviews_embeddings, query_embedding).detach().numpy()
#     reviews_similarity = np.dot(reviews_embeddings, reviews_embeddings.T)


#     keywords_idx = [np.argmax(reviews_query_similarity)]
#     mmr_ranks = [np.max(reviews_query_similarity)]

#     candidates_idx = [i for i in range(len(reviews)) if i != keywords_idx[0]]

#     for _ in range(top_n - 1):
#         candidate_similarities = reviews_query_similarity[candidates_idx, :]
#         target_similarities = np.max(reviews_similarity[candidates_idx][:, keywords_idx], axis=1)


#         mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
#         mmr_value = np.max(mmr)
#         mmr_idx = candidates_idx[np.argmax(mmr)]


#         keywords_idx.append(mmr_idx)
#         mmr_ranks.append(mmr_value)
#         candidates_idx.remove(mmr_idx)

#     output_reviews = {}
#     for i in range(len(keywords_idx)):
#         text = reviews[keywords_idx[i]]
#         output_reviews[text] = mmr_ranks[i]

#     return output_reviews
